In [1]:
PB: 0.554455

<h1>Single model submission template.

<p>every model to be used must satisfy the APIs requirements of model_template.py, and must pass all the tests like test_model_template.py

In [ ]:
"""
This is a template for the APIs of models to be used into the stacking framework.
run with Python 3.x
"""
try:
    from time import time, ctime
    import lightgbm as lgb
    import pandas as pd
    from matplotlib import pyplot as plt
    from datetime import datetime
except:
    exit("ImportError: requirements -lightgbm, -pandas, -matplotlib")


def sigma_score(preds, valid_data):
    """
    this is a custom metric used to train the model_lgbm_baseline
    """
    df_time = valid_data.params['extra_time'] # will be injected afterwards
    labels = valid_data.get_label()

    #    assert len(labels) == len(df_time)

    x_t = preds * labels #  * df_valid['universe'] -> Here we take out the 'universe' term because we already keep only those equals to 1.

    # Here we take advantage of the fact that `labels` (used to calculate `x_t`)
    # is a pd.Series and call `group_by`
    x_t_sum = x_t.groupby(df_time).sum()
    score = x_t_sum.mean() / x_t_sum.std()

    return 'sigma_score', score, True

class model_lgbm_baseline():
    """this is a baseline lightLGB model with simple features

    this class is for a model (that can also be
    a combination of bagged models)
    The commonality of the bagged models is that
    they share the feature generation
    """

    def __init__(self, name):
        self.name             = name
        self.model            = None
        self.type             = lgb.Booster
        self.training_results = None
        print("\ninit model {}".format(self.name))

    def _generate_features(self, market_data, news_data, verbose=False):
        """
        GENERAL:
        given the original market_data and news_data
        generate new features, doesn't change original data.
        NOTE: data cleaning and preprocessing is not here,
        here is only feats engineering

        MODEL SPECIFIC:
        as as a baseline for decision trees model we add
        features that are the most popular among public
        kernels on Kaggle:

        - [36] short-term lagged features on returns
        - [6]  long-term moving averages
        - [1]  day of the week

        Args:
            [market_train_df, news_train_df]: pandas.DataFrame
        Returns:
            complete_features: pandas.DataFrame
        """
        start_time = time()
        if verbose: print("Starting features generation for model {}, {}".format(self.name, ctime()))

        complete_features = market_data.copy()

        # [36] short-term lagged features on returns
        for feature in ['returnsClosePrevRaw10','returnsOpenPrevRaw10','returnsClosePrevMktres10','returnsOpenPrevMktres10']:
            for lag in [3,7,14]:
                complete_features['lag_{}_{}_max'.format(lag, feature)]  = complete_features[feature].rolling(lag, min_periods=1).max()
                complete_features['lag_{}_{}_min'.format(lag, feature)]  = complete_features[feature].rolling(lag, min_periods=1).min()
                complete_features['lag_{}_{}_mean'.format(lag, feature)] = complete_features[feature].rolling(lag, min_periods=1).mean()

        # [6]  long-term moving averages
        for feature in ['open','close']:
            for lag in [50, 100, 200]:
                complete_features['lag_{}_{}_mean'.format(lag, feature)]  = complete_features[feature].rolling(lag, min_periods=1).mean()

        self.max_lag = 200

        # [1]  day of the week
        #if type(complete_features['time'][0]) == pd._libs.tslibs.timestamps.Timestamp:
        try:    # this is Kaggle environment
            complete_features['weekday'] = complete_features['time'].apply(lambda x: x.dayofweek)
        except: # in test environment 'time' got to converted to str so need formatting
            complete_features['weekday'] = complete_features['time'].apply(lambda x: datetime.strptime(x.split()[0], "%Y-%M-%d").weekday())



        complete_features.drop(['time','assetCode','assetName'],axis=1,inplace=True)
        complete_features.fillna(0, inplace=True) # TODO: for next models control this fillna with EDA

        if verbose: print("Finished features generation for model {}, TIME {}".format(self.name, time()-start_time))
        return complete_features

    def train(self, X, Y, verbose=False):
        """
        GENERAL:
        basic method to train a model with given data
        model will be inside self.model after training

        MODEL SPECIFIC:

        - split 0.8 train validation
        - universe filter on validation
        - custom metric used (sigma_scored) ,
            need to put 'metric':'None' in parameters
        - one single lgbm with params_1 from script 67

        Args:
            X: [market_train_df, news_train_df]
            Y: [target]
            verbose: (bool)
        Returns:
            (optional) training_results
        """
        start_time = time()
        if verbose: print("Starting training for model {}, {}".format(self.name, ctime()))

        if 'returnsOpenNextMktres10' in X[0].columns:
            X[0].drop(['returnsOpenNextMktres10'],axis=1,inplace=True)

        time_reference = X[0]['time'] #time is dropped in preprocessing, but is needed later for metrics eval

        X = self._generate_features(X[0], X[1], verbose=verbose)

        # split X in X_train and Y_val
        split = int(len(X) * 0.8)
        test_train_distsance = 0
        X_train, X_val = X[:split - test_train_distsance], X[split:]
        Y_train, Y_val = Y[:split - test_train_distsance], Y[split:]

        if verbose: print("X_train shape {}".format(X_train.shape))
        if verbose: print("X_val shape {}".format(X_train.shape))

        # universe filtering on validation set
        universe_filter = X['universe'][split:] == 1.0
        X_val = X_val[universe_filter]
        Y_val = Y_val[universe_filter]

        # this is a time_val series used to calc the sigma_score later, applied split and universe filter
        time_val = time_reference[split:][universe_filter]
        assert len(time_val) == len(X_val)
        time_train = time_reference[:split - test_train_distsance]
        assert len(time_train) == len(X_train)

        # train parameters prearation
        train_cols = X.columns.tolist()
        assert 'returnsOpenNextMktres10' not in train_cols
        lgb_train = lgb.Dataset(X_train.values, Y_train, feature_name=train_cols, free_raw_data=False)
        lgb_val = lgb.Dataset(X_val.values, Y_val, feature_name=train_cols, free_raw_data=False)

        lgb_train.params = {
            'extra_time' : time_train.factorize()[0]
        }
        lgb_val.params = {
            'extra_time' : time_val.factorize()[0]
        }

        x_1 = [0.19000424246380565, 2452, 212, 328, 202]
        params_1 = {
            # from script 67
            'task': 'train',
            'boosting_type': 'gbdt',
            'objective': 'regression_l1',
            # 'objective': 'regression',
            'learning_rate': x_1[0],
            'num_leaves': x_1[1],
            'min_data_in_leaf': x_1[2],
            # 'num_iteration': x_1[3],
            'num_iteration': 239,
            'max_bin': x_1[4],
            'lambda_l1': 0.0,
            'lambda_l2' : 1.0,
            'verbose': 1
        }


        training_results = {}
        # start training
        self.model = lgb.train(
                params_1,
                lgb_train,
                num_boost_round=1000,
                valid_sets=(lgb_val,lgb_train),
                valid_names=('valid','train'),
                verbose_eval=25,
                early_stopping_rounds=20,
                #feval=sigma_score,
                evals_result=training_results)
        del X, X_train, X_val

        if verbose: print("Finished training for model {}, TIME {}".format(self.name, time()-start_time))

        self.training_results = training_results
        return training_results


    def predict(self, X, verbose=False):
        """
        given a block of X features gives prediction for everyrow

        Args:
            X: [market_train_df, news_train_df]
        Returns:
            y: pandas.Series
        """
        start_time = time()
        if verbose: print("Starting prediction for model {}, {}".format(self.name, ctime()))
        if self.model is None:
            raise "Error: model is not trained!"

        X_test = self._generate_features(X[0], X[1], verbose=verbose)
        if verbose: print("X_test shape {}".format(X_test.shape))
        y_test = self.model.predict(X_test)

        if verbose: print("Finished prediction for model {}, TIME {}".format(self.name, time()-start_time))
        return y_test


    def predict_rolling(self, historical_df, prediction_length, verbose=False):
        """
        predict features from X, uses historical for (lagged) feature generation
        to be used with rolling prediciton structure from competition

        Args:
            historical_df: [market_train_df, news_train_df]
            prediction_length: generate features on historical_df, predict only on the last rows
        """
        start_time = time()
        if verbose: print("Starting rolled prediction for model {}, {}".format(self.name, ctime()))

        processed_historical_df = self._generate_features(historical_df[0], historical_df[1], verbose=verbose)
        X_test = processed_historical_df.iloc[-prediction_length:]
        if verbose: print("X_test shape {}".format(X_test.shape))
        y_test = self.model.predict(X_test)

        if verbose: print("Finished rolled prediction for model {}, TIME {}".format(self.name, time()-start_time))
        return y_test

    def inspect(self):
        """
        visualize and examine the training of the model

        MODEL SPECIFIC:
        plots training results and feature importance
        """
        if not self.training_results:
            print("Error: No training results available")
        else:
            print("printing training results..")
            for _label, key in self.training_results.items():
                for label, result in key.items():
                    plt.plot(result,label=_label+" "+label)
            plt.title("Training results")
            plt.legend()
            plt.show()

        if not self.model:
            print("Error: No model available")
        else:
            print("printing feature importance..")
            f=lgb.plot_importance(self.model)
            f.figure.set_size_inches(10, 30)
            plt.show()
            pass

<h1>Get data

In [ ]:
from kaggle.competitions import twosigmanews
# You can only call make_env() once, so don't lose it!
env = twosigmanews.make_env()

In [ ]:
(market_train_df, news_train_df) = env.get_training_data()

<h1>`Datacleaning and preprocessing procedure

datacleaning will applied to the whole dataset for every model, the only requirements is that at the end of the procedure ***NO NEW FEATURES can be added here***. They must be added inside the feature generation section of the model

In [ ]:
import pandas as pd
import numpy as np
def prepare_predictions(market_obs_df):
    market_obs_df['close_to_open'] =  np.abs(market_obs_df['close'] / market_obs_df['open'])
    market_obs_df['assetName_mean_open'] = market_obs_df.groupby('assetName')['open'].transform('mean')
    market_obs_df['assetName_mean_close'] = market_obs_df.groupby('assetName')['close'].transform('mean')

    # if open price is too far from mean open price for this company, replace it. Otherwise replace close price.
    for i, row in market_obs_df.loc[market_obs_df['close_to_open'] >= 2].iterrows():
        if np.abs(row['assetName_mean_open'] - row['open']) > np.abs(row['assetName_mean_close'] - row['close']):
            market_obs_df.iloc[i,5] = row['assetName_mean_open']
        else:
            market_obs_df.iloc[i,4] = row['assetName_mean_close']

    for i, row in market_obs_df.loc[market_obs_df['close_to_open'] <= 0.5].iterrows():
        if np.abs(row['assetName_mean_open'] - row['open']) > np.abs(row['assetName_mean_close'] - row['close']):
            market_obs_df.iloc[i,5] = row['assetName_mean_open']
        else:
            market_obs_df.iloc[i,4] = row['assetName_mean_close']
            
    return market_obs_df.drop(['assetName_mean_open', 'assetName_mean_close','close_to_open'],axis=1)

In [ ]:
market_train_df = prepare_predictions(market_train_df)
market_train_df = market_train_df.loc[market_train_df['time'] >= '2010-01-01 22:00:00+0000']

In [ ]:
bottom, top = market_train_df.returnsOpenNextMktres10.quantile(0.001), market_train_df.returnsOpenNextMktres10.quantile(0.999)
market_train_df.returnsOpenNextMktres10 = market_train_df.returnsOpenNextMktres10.clip(bottom, top)

<h1>Initialize and train model

In [ ]:
model = model_lgbm_baseline('lgbm_baseline+logloss')

In [ ]:
target = market_train_df.returnsOpenNextMktres10
market_train_df.drop('returnsOpenNextMktres10', axis=1, inplace=True)

In [ ]:
model.train([market_train_df, news_train_df], target, verbose=True)

In [ ]:
model.inspect()

<h1>Prediction loop

In [ ]:
days = env.get_prediction_days()

In [ ]:
# skip a prediction (for testing)
#env.predict(predictions_template_df)

In [ ]:
from time import time, ctime

total_market_df = pd.DataFrame(columns=['time', 'assetCode', 'assetName', 'volume', 'close', 'open',
       'returnsClosePrevRaw1', 'returnsOpenPrevRaw1',
       'returnsClosePrevMktres1', 'returnsOpenPrevMktres1',
       'returnsClosePrevRaw10', 'returnsOpenPrevRaw10',
       'returnsClosePrevMktres10', 'returnsOpenPrevMktres10','period', 'universe'])

max_lag, days_count = model.max_lag, 0
for (market_obs_df, news_obs_df, predictions_template_df) in days:
    days_count += 1
    if not days_count%50: print(days_count)
        
    market_obs_df['period']   = days_count
    market_obs_df['universe'] = 1
    
    start_time = time()
    total_market_df = pd.concat([total_market_df, market_obs_df])
    #total_news_obs_df.append(news_obs_df)
        
    history_market_df = total_market_df[total_market_df['period'] > days_count - max_lag - 1].drop('period', axis=1)
    predictions = model.predict_rolling([history_market_df, None],
                                        len(predictions_template_df), verbose=True)
    
    predictions_template_df.confidenceValue = predictions.clip(-1, 1)
    env.predict(predictions_template_df)
    print("[{}] loop prediction {}, TIME {}".format(days_count, ctime(), time()-start_time))
print('Done!')

## **`write_submission_file`** function

Writes your predictions to a CSV file (`submission.csv`) in the current working directory.

In [ ]:
env.write_submission_file()

In [ ]:
# We've got a submission file!
import os
print([filename for filename in os.listdir('.') if '.csv' in filename])

As indicated by the helper message, calling `write_submission_file` on its own does **not** make a submission to the competition.  It merely tells the module to write the `submission.csv` file as part of the Kernel's output.  To make a submission to the competition, you'll have to **Commit** your Kernel and find the generated `submission.csv` file in that Kernel Version's Output tab (note this is _outside_ of the Kernel Editor), then click "Submit to Competition".  When we re-run your Kernel during Stage Two, we will run the Kernel Version (generated when you hit "Commit") linked to your chosen Submission.

## Restart the Kernel to run your code again
In order to combat cheating, you are only allowed to call `make_env` or iterate through `get_prediction_days` once per Kernel run.  However, while you're iterating on your model it's reasonable to try something out, change the model a bit, and try it again.  Unfortunately, if you try to simply re-run the code, or even refresh the browser page, you'll still be running on the same Kernel execution session you had been running before, and the `twosigmanews` module will still throw errors.  To get around this, you need to explicitly restart your Kernel execution session, which you can do by pressing the Restart button in the Kernel Editor's bottom Console tab:
![Restart button](https://i.imgur.com/hudu8jF.png)